# Generalized Advantage Estimation

## 목적

- 현재 policy gradient method의 목적은 누적 discount reward를 optimization하는 것이다. 하지만 학습할 때 많은 양의 sample이 필요하고 sample data가 nonstationarity임에도 stable한 학습이 어렵다.
- 따라서, \\(TD(\lambda)\\)와 같은 advantage function의 exponentially-weighted estimator를 사용하여 policy gradient estimate의 variance를 줄인다. variance를 줄이게 되면 stable하게 학습을 할 수 있다.
- 또한, policy와 value function에 대한 Trust Region optimization(앞선 TRPO에서 소개되었다.) 사용

## 요약

- 따라서, advantage function의 good estimate을 얻는 variance reduction에 대해서 알아보자.
  - 위 문제를 해결하기 위해서, Generalized advantage estimator를 사용했고, 이것은 bias-variance tradeoff를 조절하는 두개의 parameter \\(\gamma, \lambda\\)가 있다.
  - 그리고, Trust region policy optimization과 value function을 optimize하는 trust region 알고리즘의 아이디어를 어떻게 합치는지 증명하였다.

- REINFORCE, expected total returns의 gradient에 대한 unbiased estimate을 REINFORCE라 한다. 하지만 하나의 에피소드가 끝날 때 update를 진행하는 방식이므로 높은 variance를 가지게 된다.
- Actor-critic, 하나의 value function을 사용한다. 그리고 bias하고 낮은 variance를 가진다. 만약 높은 variance를 가진다면 sampling을 더 하면 되는데 bias한 경우는 치명적이다. bias한 경우는 수렴이 되지 않거나 local optimum이 아닌 poor solution에 수렴하도록 만든다.

## 준비